In [1]:
with open('FinalProject/wav2vec-kenlm/data/mesivta/train.txt', 'r') as f:
    text = f.readlines()
    text = ' '.join(text).replace('\n', '')
letters = set()
for line in text:
    for letter in line:
        letters.add(letter)

In [2]:
vocab_dict = {v: k for k, v in enumerate(sorted(letters))}
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
vocab_dict['<s>'] = len(vocab_dict)
vocab_dict['</s>'] = len(vocab_dict)
len(vocab_dict)

32

In [3]:
# import json
# path = "/teamspace/studios/this_studio/models/wav2vec2BertLm"
# with open(path + '/vocab.json', 'w') as vocab_file:
#     json.dump(vocab_dict, vocab_file)

In [4]:
path= '/teamspace/studios/this_studio/models/facebook/w2v-bert-2.0-finetuned'

In [5]:
from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(path, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [6]:
from transformers import SeamlessM4TFeatureExtractor

feature_extractor = SeamlessM4TFeatureExtractor.from_pretrained(path)

In [7]:
from transformers import Wav2Vec2BertProcessor

processor = Wav2Vec2BertProcessor.from_pretrained(path,unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
# processor.save_pretrained('models/wav2vec2bert')

In [8]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(vocab_dict.keys()),
    kenlm_model_path="/teamspace/studios/this_studio/FinalProject/wav2vec-kenlm/data/mesivta/5gram.arpa",
)

Loading the LM will be faster if you build a binary file.
Reading /teamspace/studios/this_studio/FinalProject/wav2vec-kenlm/data/mesivta/5gram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


In [14]:
len(vocab_dict.keys())

dict_keys(['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'ך', 'כ', 'ל', 'ם', 'מ', 'ן', 'נ', 'ס', 'ע', 'ף', 'פ', 'ץ', 'צ', 'ק', 'ר', 'ש', 'ת', '|', '[UNK]', '[PAD]', '<s>', '</s>'])

In [9]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

In [10]:
processor_with_lm.save_pretrained('models/wav2vec2bertLm')